In [6]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [7]:
background = None
accumulated_weight = 0.5

# Defining ROI coordinates where hand will be evaluated
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

# Get the accumulated average of the ROI (if it changes, then we know hand has entered the ROI)
def calc_accum_avg(frame,accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background,accumulated_weight)

In [8]:
# Using thresholding to grab the hand segment form ROI
def segment(frame,threshold_min=25):
#     Getting difference between current frame and the previous background
    diff = cv2.absdiff(background.astype('uint8'), frame)
    ret,thresholded = cv2.threshold(diff,threshold_min,255, cv2.THRESH_BINARY)
    
#     Getting contours of the ROI 
    image, contours, hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
#         Assuming the largest external contour in ROI is the hand
        hand_segment = max(contours,key=cv2.contourArea)
    
    return (thresholded,hand_segment)

In [9]:
# Using a Convex Hull, which draws a ploygon by connecting points around the most external points in a frame
def count_fingers(thresholded, hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    
#    Calculating edges of figure
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    center_x = (left[0] + right[0]) // 2
    center_y = (top[1] + bottom[1]) // 2
    
    distance = pairwise.euclidean_distances([(center_x,center_y)],Y=[left,right,top,bottom])[0]
    
#     Creating circle with 90% radius of euclidean distance (this distance being of the point furthest to the center)
    max_distance = distance.max()
    radius = int(0.8*max_distance)
    circumference = 2*np.pi*radius
    
#     Creating circular roi for the palm
    circular_roi = np.zeros(thresholded.shape[:2],dtype='uint8')
    cv2.circle(circular_roi,(center_x,center_y),radius,255,10)
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
#     Finding contours of said area to then use it as point of reference to know if finger is extended or not
    image,contours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    count = 0
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        
#         Checking that points are not part of the wrist, assuming that the wrist is below 
        out_of_wrist = ((center_y + (center_y*0.25)) > (y + h))
    
#     Checking that points are not background noise (by checking that point is within the limited range from circular_roi)
        limit_points = ((circumference*0.25) > contour.shape[0])
      
        if out_of_wrist and limit_points:
            count += 1
        
    return count

In [10]:
cam = cv2.VideoCapture(0)
num_frames = 0

while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    
#     Setting ROI
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
#     Applying blur for better contour detection
    gray = cv2.GaussianBlur(gray,(7,7),0)

    if num_frames < 60:
        #     Buffering accumulated weight from background
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy,'LOADING BACKGROUND...',(200,400),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)  
            cv2.imshow('Finger Counter', frame_copy)
    else:
        hand = segment(gray)
        if hand is not None:
            thresholded, hand_segment = hand
            
#             Drawing contours around the hand, counting and showing amount of fingers extended
            cv2.drawContours(frame_copy,[hand_segment + (roi_right, roi_top)],-1,(0,0,255),1)
            fingers = count_fingers(thresholded,hand_segment)
            cv2.putText(frame_copy, str(fingers),(200,400),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
#             For debugging
            cv2.imshow('Thresholded', thresholded)
    
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    num_frames +=1
    
    cv2.imshow('Finger Counter', frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
    

-1